# Lesson 3: Enable Logging

#### Import all needed packages

In [ ]:
import boto3
import json
import os

In [ ]:
# Creating Amazon Bedrock Client
# Note- Not Bedrock_Runtime_client

bedrock_client = boto3.client('bedrock', region_name="us-east-1")

In [ ]:
# Choose "CloudWatchHelper" as a service bcz Cloudwatch is used to gets Logs into AWS Services

from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [ ]:
#Create a Log-Groups and select where we want to save our Log files
#Before that Plz Check ur IAM Policy that aloows to Action: "CreateLogStream" and "PutLogEvents"
log_group_name = '/mera/amazon/bedrock/logs'
cloudwatch.create_log_group(log_group_name)   #call here CloudWatch_Helper function

Lets Configure Amazon Bedrock to send those Cloudwatch Logs ------>

In [ ]:
# Create a dict of parameters

mera_loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['mera_LOGGINGROLEARN'],
        #Let's handel the large logs generations, so need to give here ana s3 output Location for lager Log files/Image/Text. ect
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['mera_LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    #Since, we handeled Large logs, lets store anything output in S3 bucket thats different from Logs, that might be any other file/text/image gereted by LLM Models
    's3Config': {
        'bucketName': os.environ['mera_LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,  #bcz here particularly, Mai text data hi manage kr rha hu
}

In [ ]:
bedrock_client.put_model_invocation_logging_configuration(loggingConfig=mera_loggingConfig)

In [ ]:
bedrock_client.get_model_invocation_logging_configuration()

In [ ]:
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [ ]:
prompt = "Write an crisp article about the Indian Parliamentary Model and why its better than other Nations."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

#Fine tune ur prompt and change Temp and topP.. and play around it.

In [ ]:
response = bedrock_runtime_client.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)

In [ ]:
# Let's pull up all logs of recent and print

cloudwatch.print_recent_logs(log_group_name)

U can further re-verify into Console >> CloudWatch >> Log Groups(Left Navigation bar) >> Bedrock Logs >> Log Streams >> particular Log

In [ ]:
# from IPython.display import HTML
# aws_url = os.environ['AWS_CONSOLE_URL']

In [ ]:
# HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')


END